In [ ]:
#ollama serve
#ollama run qwen2.5:0.5b

In [ ]:
from llama_index.llms.ollama import Ollama
from llama_index.core import Settings

# low hardware requirement llm - for testing
llm = Ollama(model="qwen2.5:0.5b", request_timeout=120.0)
Settings.llm = llm

In [ ]:
from llama_index.tools.mcp import BasicMCPClient, McpToolSpec

mcp_client = BasicMCPClient("http://mcp-service:8000/sse")
mcp_tools = McpToolSpec(client=mcp_client)

In [ ]:
tools = await mcp_tools.to_tool_list_async()
for tool in tools:
    print(tool.metadata.name, tool.metadata.description)

In [ ]:
from llama_index.core.agent.workflow import FunctionAgent

SYSTEM_PROMPT = "You are a helpful assistant that can use tools to answer questions."

async def get_agent():
    tool_list = await mcp_tools.to_tool_list_async()
    return FunctionAgent(
        name="QwenbAgent",
        description="Agent using qwen2.5:0.5b and MCP tools",
        tools=tool_list,
        llm=llm,
        system_prompt=SYSTEM_PROMPT
    )

agent = await get_agent()

In [ ]:
from llama_index.core.workflow import Context

ctx = Context(agent)

async def run_agent():
    message = "Say hello"
    response = await agent.run(message, ctx=ctx)
    print("Agent:", response)

await run_agent()

In [ ]:
await agent.run("Please use the echo tool to repeat the message 'Testing 123'", ctx=ctx)